In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

In [ ]:
!pip install nilearn

In [ ]:
# 健常者グループの被験者IDのリスト
healthy_subject_ids = ['4101', '4103', '4106', '4107', '4109', '4110', '4111', '4112', '4113', '4114', '4115', '4116', '4117', '4118', '4119', '4120', '4121', '4122', '4123', '4125', '4126', '4128', '4129', '4130', '4131']

# パーキンソン病患者グループの被験者IDのリスト
patient_subject_ids = ['4201', '4204', '4205', '4206', '4207', '4208', '4210', '4211', '4212', '4213', '4214', '4215', '4217', '4218', '4219', '4220', '4221', '4224', '4225', '4226', '4227']

In [ ]:
# sessionのリスト
ses_ids = ['1', '2']

In [ ]:
# runsのリスト
run_ids = ['1', '2', '3', '4', '5', '6']

In [ ]:
vs_titles = ["alerting", "executive", "orienting"]

In [ ]:
from nilearn.glm.second_level import SecondLevelModel
import random

def second_level_glm(vs_title):
  contrast_maps = []
  for subject_id in healthy_subject_ids + patient_subject_ids:
    for ses_id in ses_ids:
      for run_id in run_ids:
        contrast_maps.append(f'/content/drive/MyDrive/ANT/{vs_title}/sub-RC{subject_id}/ses-{ses_id}/sub-RC{subject_id}_ses-{ses_id}_task-ANT_run-{run_id}_bold.nii.gz')

  # 健常者は0、パーキンソン病患者は1
  group_labels = [0]*len(healthy_subject_ids)*len(ses_ids)*len(run_ids) + [1]*len(patient_subject_ids)*len(ses_ids)*len(run_ids)
  design_matrix = pd.DataFrame({'intercept': np.ones(len(group_labels)), 'group': group_labels})

  # 第二レベルモデルの初期化とフィッティング
  second_level_model = SecondLevelModel(smoothing_fwhm=8)
  second_level_model = second_level_model.fit(contrast_maps, design_matrix=design_matrix)

  # 統計マップの計算
  z_map = second_level_model.compute_contrast('group', output_type='z_score')

  # Bonferroni補正
  threshold = -np.log10(0.05 / z_map.get_data().size)

  return z_map, threshold

In [ ]:
from nilearn.plotting import plot_stat_map

for vs_title in vs_titles:
  z_map, threshold = second_level_glm(vs_title)
  plot_stat_map(z_map, threshold=threshold, display_mode='z', cut_coords=5, black_bg=True, title=f'Group level contrast - {vs_title}')